In [14]:
import os
import rioxarray as rxr
import xarray as xr
from dataclasses import asdict
from dscreator.cfarray.attributes import DatasetAttrsGrid, VariableAttrs
from datetime import datetime
from dscreator import utils

The rapport can be found [here](https://niva365.sharepoint.com/sites/projects/3383/Shared%20Documents/Forms/AllItems.aspx?csf=1&web=1&e=A7HwOO&CT=1678890409578&OR=OWA%2DNT&CID=4ae0ac88%2Ddc20%2D15e6%2D4318%2D5052c4a4fa9e&FolderCTID=0x01200023B9EEB8E9C0094580920107C7844708&id=%2Fsites%2Fprojects%2F3383%2FShared%20Documents%2FProjects%2F2017%20T%C3%A5legrenseprosjekt%2FRapport&viewid=aa29078f%2D5cb7%2D41dd%2Da376%2D38bbe9dc6fa4)
Notes:

* The Steady-State Water Chemistry (SSWC), som beregner tålegrenser for sterk syre og overskridelse med utgangspunkt i at utlekkingen av nitrogen forblir konstant på dagens nivå.  
* The First-order Acidity Balance (FAB), calculates limits for sulphur og nitrogen

In [33]:
da_fab_tif = rxr.open_rasterio(os.path.join("data", "fab_exceed_ns_12_16_meq1.tif"))
da_fab_tif.name = "fab"
da_fab_tif.attrs.update(asdict(
    VariableAttrs(long_name="First-order Acidity Balance", units="mEkv/m^2/year", short_name="FAB")
))
da_fab_tif.rio.reproject("EPSG:4326")
da_sswc_tif = rxr.open_rasterio(os.path.join("data", "sswc_exceed_12_16_meq1.tif"))
da_sswc_tif.name = "sswc"
da_sswc_tif.attrs.update(asdict(
    VariableAttrs(long_name="Steady-State Water Chemistry ", units="mEkv/m^2/year", short_name="sswc")
))
da_sswc_tif.rio.reproject("EPSG:4326")

<xarray.DataArray 'sswc' (band: 1, y: 8628, x: 19821)>
array([[[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]]], dtype=float32)
Coordinates:
  * x            (x) float64 -2.642 -2.64 -2.638 -2.636 ... 33.28 33.29 33.29
  * y            (y) float64 72.55 72.55 72.54 72.54 ... 56.91 56.91 56.91 56.91
  * band         (band) int64 1
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:       Area
    DataType:            Generic
    RepresentationType:  ATHEMATIC
    scale_factor:        1.0
    add_offset:          0.0
    long_name:           Steady-State Water Chemistry 
    units:               mEkv/m^2/year
    short_name:          sswc
    _FillValue:          -1.0

In [34]:
ds = xr.merge([da_fab_tif, da_sswc_tif])



In [35]:
ds.attrs = DatasetAttrsGrid(
            title="Test limits",
            title_no="Test grenser",
            summary="Test limits summary",
            summary_no="Test grenser oppsummering",
            keywords=",".join(
                [
                    "GCMDSK:EARTH SCIENCE > LAND SURFACE > SOILS > NITROGEN",
                    "GCMDSK:EARTH SCIENCE > LAND SURFACE > SOILS > SULFUR",
                    "GCMDLOC:CONTINENT > EUROPE > NORTHERN EUROPE > SCANDINAVIA > NORWAY",
                ]
            ),
            keywords_vocabulary=",".join(
                [
                    "GCMDSK:GCMD Science Keywords:https://gcmd.earthdata.nasa.gov/kms/concepts/concept_scheme/sciencekeywords",
                    "GCMDLOC:GCMD Locations:https://gcmd.earthdata.nasa.gov/kms/concepts/concept_scheme/locations",
                ]
            ),
            iso_topic_category="environment",
            spatial_representation="grid",
            date_created=str(datetime.now()),
            project="Test Limits",
            time_coverage_start=utils.to_isoformat(datetime(2012, 1, 1)),
            time_coverage_end=utils.to_isoformat(datetime(2012, 1, 1)),
            geospatial_lon_min=float(ds.x.min()),
            geospatial_lon_max=float(ds.x.max()),
            geospatial_lat_min=float(ds.y.min()),
            geospatial_lat_max=float(ds.y.max()),
        )

AttributeError: 'Dataset' object has no attribute 'latitude'

<xarray.DataArray 'y' ()>
array(6355520.507609)
Coordinates:
    spatial_ref  int64 0

<xarray.DataArray 'fab' (band: 1, y: 14122, x: 10349)>
[146148578 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -1.09e+05 -1.089e+05 ... 1.133e+06 1.133e+06
  * y            (y) float64 8.05e+06 8.05e+06 8.05e+06 ... 6.356e+06 6.356e+06
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:       Area
    DataType:            Generic
    RepresentationType:  ATHEMATIC
    _FillValue:          -1.0
    scale_factor:        1.0
    add_offset:          0.0
    long_name:           First-order Acidity Balance
    units:               mEkv/m^2/year
    short_name:          FAB

In [22]:
ds_nilu = xr.open_dataset(os.path.join("data", "fab_proj.tif"))

In [23]:
ds_nilu

<xarray.Dataset>
Dimensions:      (band: 1, x: 14192, y: 14600)
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -2.641 -2.639 -2.636 -2.634 ... 33.28 33.29 33.29
  * y            (y) float64 72.55 72.55 72.55 72.54 ... 56.91 56.91 56.91 56.91
    spatial_ref  int64 ...
Data variables:
    band_data    (band, y, x) float32 ...

In [24]:
da_fab = rxr.open_rasterio(os.path.join("data", "fab_proj.tif"))

In [25]:
da_fab

<xarray.DataArray (band: 1, y: 14600, x: 14192)>
[207203200 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -2.641 -2.639 -2.636 -2.634 ... 33.28 33.29 33.29
  * y            (y) float64 72.55 72.55 72.55 72.54 ... 56.91 56.91 56.91 56.91
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     -1.0
    scale_factor:   1.0
    add_offset:     0.0